In [1]:
import os
#comment this if you are not using puffer?
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader,SubsetRandomSampler
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from common import setup_dataflow,vali
import wandb
wandb.login()

/home/nutapolt/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nutapol-1997. Use `wandb login --relogin` to force relogin


True

In [3]:
torch.cuda.is_available()

True

In [4]:
from sklearn.model_selection import KFold
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage

In [5]:
class EEG:
    def __init__(self, path, base_url, subjects, runs):
        self.subpath = ''
        self.path = path
        print(path)
        self.base_url = base_url
        self.subjects = subjects
        self.runs = runs
        
        # download data if does not exist in path.
        # self.load_data()
        self.data_to_raw()
    
    def load_data(self):
        print(f">>> Start download from: {self.base_url}.")
        print(f"Downloading files to: {self.path}.")
        for subject in self.subjects:
            eegbci.load_data(subject,self.runs,path=self.path,base_url=self.base_url)
        print("Done.")
    
    
        
        print("Done.")
        return self.raw
    def filter(self, freq):
        raw = self.raw
        low, high = freq
        print(f">>> Apply filter.")
        self.raw.filter(low, high, fir_design='firwin', verbose=20)
        return  raw
    def raw_ica(self):
        raw = self.raw
        ica = mne.preprocessing.ICA(n_components=1, max_iter=100)
        ica.fit(raw)
        ica.exclude = [1, 2]  # details on how we picked these are omitted here
        ica.plot_properties(raw, picks=ica.exclude)
        ica.apply(raw)
        print('ICA DONE ????')
        return  raw
        
    def get_events(self):
        event_id = dict(T1=0, T2=1) # the events we want to extract
        events, event_id = mne.events_from_annotations(self.raw, event_id=event_id)
        return events, event_id
    
    def get_epochs(self, events, event_id):
        picks = mne.pick_types(self.raw.info, eeg=True, exclude='bads')
        tmin = 0
        tmax = 4
        epochs = mne.Epochs(self.raw, events, event_id, tmin, tmax, proj=True, 
                            picks=picks, baseline=None, preload=True)
        return epochs
    
    
        
    
    def data_to_raw(self):
        fullpath = os.path.join(self.path, *self.subpath.split(sep='/'))
        #print(f">>> Extract all subjects from: {fullpath}.")
        extension = "fif"
        raws = []
        count = 1
        for i, subject in enumerate(self.subjects):
            sname = f"S{str(subject).zfill(3)}".upper()
            
            for j, run in enumerate(self.runs):
                rname = f"{sname}R{str(run).zfill(2)}".upper()
                path_file = os.path.join(fullpath, sname, f'{rname}.{extension}')
                #print(path_file)
                #print(f"Loading file #{count}/{len(self.subjects)*len(self.runs)}: {f'{rname}.{extension}'}")
                raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
                raws.append(raw)
                count += 1

        raw = mne.io.concatenate_raws(raws)
        eegbci.standardize(raw)
        montage = mne.channels.make_standard_montage('standard_1005')
        raw.set_montage(montage)
        self.raw = raw
    
    def create_epochs(self):
        print(">>> Create Epochs.")
        
        events, event_id = self.get_events()
        self.epochs = self.get_epochs(events, event_id)
        print("Done.")
        return events , event_id
# getepoch(raw,4, 10,reject_bad=False,on_missing='warn')    
    def get_X_y(self):
        events = mne.find_events(raw)
        
        epochs = mne.Epochs(
        raw,
        events,
        event_id=[1,2,3],
        tmin=0,
        tmax=5,
        picks="data",
        on_missing='warn',
        baseline=None
    )
            
            #events , event_id=self.create_epochs()
        self.X = epochs.get_data()
        self.y = epochs.events[:, -1]
        return self.X, self.y 

        
def getepoch(raws,trial_duration, calibration_duration,reject_bad=False,on_missing='warn'):
    #reject_criteria = dict(eeg=100e-6)  # 100 µV
    #flat_criteria = dict(eeg=1e-6)  # 1 µV
    epochs_list = []
    raws = [raws]
    print(len(raws))
    for raw in raws:
        print(raw)
        events = mne.find_events(raw)
        epochs = mne.Epochs(
            raw,
            events,
            event_id=[1,2,3],
            tmin=-calibration_duration,
            tmax=trial_duration,
            picks="data",
            on_missing=on_missing,
            baseline=None
        )
        
        epochs_list.append(epochs)
    epochs = mne.concatenate_epochs(epochs_list)
    labels = epochs.events[:,-1]
    
    print(f'Found {len(labels)} epochs')
    print(epochs)
    print(labels)

    return epochs.get_data(),epochs,labels

        
        
def do_plot(train_loss, valid_loss,ty):
    if ty == "loss":
        plt.figure(figsize=(10,10))
        
        plt.plot(train_loss, label='train_loss')
        plt.plot(valid_loss, label='valid_loss')
        plt.title('loss {}'.format(iter))
        plt.legend()
        plt.show()
    if ty == "acc":
        plt.figure(figsize=(10,10))
        
        plot_ty=torch.tensor(train_loss, device = 'cpu')
        plat_va=torch.tensor(valid_loss, device = 'cpu')
        plt.plot(plot_ty, label='train_acc')
        plt.plot(plat_va, label='valid_acc')
        plt.title('acc {}'.format(iter))
        plt.legend()
        plt.show()

In [6]:
# home directory + datasets folder
#path = '/content/drive/MyDrive/MNE-eegbci-data/files/eegmmidb/'
path = ''
base_url = 'https://physionet.org/files/eegmmidb/'
# subjects = [1]
#runs = [3, 4, 7, 8, 11, 12]
runs = [3, 5, 7]
subjects = [i for i in range(3,4)]
#subjects = [1]
# runs = [6,10,14]
print(path)
eeg = EEG(path, base_url, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter
freq = (1., 45.)
raw=eeg.filter(freq=freq)
#raw=eeg.data_to_raw()
print("Filter done")
#raw=eeg.raw_ica()
#eeg.create_epochs()

/tmp/ipykernel_11511/1373606278.py:71: RuntimeWarning: This filename (S003/S003R03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/tmp/ipykernel_11511/1373606278.py:71: RuntimeWarning: This filename (S003/S003R05.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/tmp/ipykernel_11511/1373606278.py:71: RuntimeWarning: This filename (S003/S003R07.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _i

Raw done
>>> Apply filter.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 sec)

Filter done


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


In [7]:
X, y = eeg.get_X_y()

print(X.shape, y.shape)
X = X[:,:,:]
X.shape

144 events found
Event IDs: [1 2 4]
Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 72 events and 1251 original time points ...
0 bad epochs dropped
(72, 2, 1251) (72,)


/tmp/ipykernel_11511/1373606278.py:92: RuntimeWarning: No matching events found for 3 (event id 3)
  epochs = mne.Epochs(


(72, 2, 1251)

In [8]:
y=y-1
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print('Train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)

Train size (64, 2, 1251) (64,)
Test size (8, 2, 1251) (8,)


In [10]:
y_test

array([1, 0, 0, 1, 0, 0, 1, 1])

In [12]:
X_tensor = torch.tensor(X_test).float()
y_tensor = torch.tensor(y_test).long()

dataset_vail = TensorDataset(X_tensor, y_tensor)
vail_loader = torch.utils.data.DataLoader(dataset_vail, batch_size=32, shuffle=True)

In [13]:
#create dataset
X_tensor = torch.tensor(X_train).float()
y_tensor = torch.tensor(y_train).long()



dataset = TensorDataset(X_tensor, y_tensor)

In [14]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(2, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(160128, 2)
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

In [15]:



wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"ourdata_CNN_2ch_2class_cross_ourdata", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.00000001,
      "architecture": "CNN",
      "dataset": "Nutapol T.",
      "epochs": 50000,
      "weightname":"ourdata_CNN_2ch_2class_cross",
       
        
      }
    )

config = wand.config
#print(config.num_step_per_epoch)

In [16]:
num_folds = 5
#splits = KFold(n_splits=num_folds,shuffle=True,random_state=42)
splits = KFold(n_splits=num_folds)

In [17]:
net = ConvNet().cuda(0)

In [18]:
for fold_idx, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
        
        X_tensor[train_idx]
        y_tensor
        print('Fold {}'.format(fold_idx + 1))
        #print('Fold {}'.format(fold_idx + 1))
        print(X_tensor[train_idx].shape)
        print(y_tensor[train_idx].shape)
        #train_loader, val_loader = setup_dataflow(dataset, train_idx, val_idx)

Fold 1
torch.Size([51, 2, 1251])
torch.Size([51])
Fold 2
torch.Size([51, 2, 1251])
torch.Size([51])
Fold 3
torch.Size([51, 2, 1251])
torch.Size([51])
Fold 4
torch.Size([51, 2, 1251])
torch.Size([51])
Fold 5
torch.Size([52, 2, 1251])
torch.Size([52])


In [ ]:
results_per_fold = []
batch_size = 32
from common import train


optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
for epoch_B in range(1):
    print("Epoch Num {}".format(epoch_B))
    for fold_idx, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

        print('Fold {}'.format(fold_idx + 1))

        train_loader, test_loader = setup_dataflow(X_tensor,y_tensor, train_idx, val_idx)
        num_step =math.ceil(len(train_loader.dataset) / batch_size)
        config.num_step_per_epoch=num_step
        train_loss,valid_loss,train_accuracy,valid_accuracy =train(
                                                                model = net,
                                                                gpu_num = 0,
                                                                train_loader = train_loader,
                                                                test_loader = test_loader,
                                                                vail_loader = vail_loader,
                                                                optimizer = optimizer  ,
                                                                criterion = criterion ,
                                                                wand = wand
                                                                     )
        results_per_fold.append([train_accuracy, valid_accuracy])
wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

Epoch Num 0
Fold 1
Val Loss: 0.6932519674301147, Val Acc: 50.0
Epoch 1/50000, Tr Loss: 0.6997, Tr Acc: 54.9020, Val Loss: 0.6968, Val Acc: 38.4615
Epoch 51/50000, Tr Loss: 0.6991, Tr Acc: 60.7843, Val Loss: 0.7038, Val Acc: 38.4615
Epoch 101/50000, Tr Loss: 0.7298, Tr Acc: 47.0588, Val Loss: 0.7066, Val Acc: 53.8462
Epoch 151/50000, Tr Loss: 0.6734, Tr Acc: 60.7843, Val Loss: 0.7077, Val Acc: 53.8462
Epoch 201/50000, Tr Loss: 0.6719, Tr Acc: 47.0588, Val Loss: 0.7086, Val Acc: 53.8462
Epoch 251/50000, Tr Loss: 0.7078, Tr Acc: 47.0588, Val Loss: 0.7098, Val Acc: 53.8462
Epoch 301/50000, Tr Loss: 0.7147, Tr Acc: 45.0980, Val Loss: 0.7109, Val Acc: 46.1538
Epoch 351/50000, Tr Loss: 0.6794, Tr Acc: 56.8627, Val Loss: 0.7119, Val Acc: 46.1538
Epoch 401/50000, Tr Loss: 0.6843, Tr Acc: 50.9804, Val Loss: 0.7130, Val Acc: 38.4615
Epoch 451/50000, Tr Loss: 0.6925, Tr Acc: 56.8627, Val Loss: 0.7141, Val Acc: 38.4615
Epoch 501/50000, Tr Loss: 0.6995, Tr Acc: 45.0980, Val Loss: 0.7151, Val Acc: 38

### VALI

In [61]:



wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"physionet_CNN_2ch_2class_cross_vail", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.00001,
      "architecture": "CNN",
      "dataset": "Nutapol T.",
      "epochs": 10,
      "weightname":"physionet_CNN_2ch_2class_cross_vail",
       
        
      }
    )

config = wand.config
#print(config.num_step_per_epoch)

Problem at: /tmp/ipykernel_10930/1186754688.py 1 <cell line: 1>


Traceback (most recent call last):
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 1043, in init
    run = wi.init()
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 553, in init
    manager._inform_init(settings=self.settings, run_id=self.settings.run_id)
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/wandb_manager.py", line 161, in _inform_init
    svc_iface._svc_inform_init(settings=settings, run_id=run_id)
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/service/service_sock.py", line 39, in _svc_inform_init
    self._sock_client.send(inform_init=inform_init)
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 140, in send
    self.send_server_request(server_req)
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 84, in send_server_request
    self._send_message(msg)
  File "/hom

Exception: problem

In [62]:
del net
net = ConvNet().cuda(0)

In [63]:
net.load_state_dict(torch.load('weight/ourdata_CNN_2ch_2class_cross'))

<All keys matched successfully>

In [64]:
criterion = nn.CrossEntropyLoss()
valid_loss,valid_accuracy =vali(
    model = net,
    gpu_num = 0,
    train_loader = dl,
    criterion = criterion ,
    wand = wand
         )

Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 62.5000
Epoch  Val Loss: 0.6250, Val Acc: 


KeyboardInterrupt

Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 81, in _send_message
    self._sendall_with_error_handle(header + data)
  File "/home/nutapolt/.local/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 61, in _sendall_with_error_handle
    sent = self._sock.send(data[total_sent:])
BrokenPipeError: [Errno 32] Broken pipe
